## Optimização de hyper-parâmetros

A ideia é que esta função selecione um número X de diferentes redes.

Depois vamos criar uma função que para cada rede, vai fazer o treino e testar o score com os dados de validação. Essa função depois ordena consoante o melhor valor de AUC.

In [1]:
from sklearn.model_selection import ParameterSampler
import random as r
def selecaoHyperParametros(d,neuronios):
    lista_parametros = list(ParameterSampler(d, n_iter=4, random_state=10))
    r.seed(10)
    for var in lista_parametros:
        var['topologia'] = r.choices(neuronios,k=var['nrCamadas'])
    return lista_parametros

In [2]:
from keras import models,layers
def criaRede(param,inputSize):
    model=models.Sequential()
    model.add(layers.Dense(param['topologia'][0],activation=param['ativacao'],input_shape=(inputSize,)))
    for var in param['topologia'][1:]:
        model.add(layers.Dense(var,activation=param['ativacao']))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer=param['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [3]:
from sklearn.metrics import roc_curve, auc
def optimizacaoHyperParametros(d,neuronios,trainX,trainY,valX,valY):
    params = selecaoHyperParametros(d,neuronios)
    for param in params:
        print(param)
        rede = criaRede(param,trainX.shape[1])
        history = rede.fit(trainX,
                                trainY,
                                epochs=param['epochs'],
                                batch_size=param['batch_size'],
                                validation_data=(valX,valY))
        pred = rede.predict(valX)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(valY, pred)
        score = auc(false_positive_rate, true_positive_rate)
        param['score'] = score
    return params

## Aqui estão os parametros que vamos optimizar

### Faltam ainda ver o dropout, early stopping e regularização

### Optimizer temos de ver os parâmetros existentes também:
- rmsprop
    - rho: float >= 0.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
- SGD
    - momentum: float >= 0. Parameter that accelerates SGD in the relevant direction and dampens oscillations.
    - decay: float >= 0. Learning rate decay over each update.
    - nesterov: boolean. Whether to apply Nesterov momentum.
- Adam
    - beta_1: float, 0 < beta < 1. Generally close to 1.
    - beta_2: float, 0 < beta < 1. Generally close to 1.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
    - amsgrad: boolean. Whether to apply the AMSGrad variant of this algorithm from the paper "On the Convergence of Adam and Beyond"
    
##### Todos têm learning rate!
##### Existem ainda mais optimizers mas não sei se vale a pena ver todos!

In [4]:
dicionario = {
    'nrCamadas':[1,2,3,4,5,6,7,8],
    'ativacao':['relu','elu','exponential','tanh','sigmoid','linear'],
    'epochs':[4,8,10,20],
    'batch_size':[64,128,256,512],
    'optimizer':['rmsprop','adam','sgd']
}
neuronios = [2,3,4,5,8,10,16,32,64,128]

In [5]:
import pandas as pd
ms = None
y = None
for df in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=60000):
    if ms is None:
        ms = df.iloc[:,0:10]
        y = df['HasDetections']
    else:
        y = pd.concat([y, df['HasDetections']], ignore_index=True)
        ms = pd.concat([ms,df.iloc[:,0:10]], ignore_index=True)
    print('Feito')

Feito
Feito
Feito
Feito
Feito
Feito
Feito
Feito
Feito
Feito


In [6]:
trainX = ms[:500000]
valX = ms[500000:550000]
trainY = y[:500000]
valY = y[500000:550000]

In [8]:
res = optimizacaoHyperParametros(dicionario,neuronios,
                          trainX,trainY,
                          valX,valY)

{'optimizer': 'sgd', 'nrCamadas': 6, 'epochs': 8, 'batch_size': 128, 'ativacao': 'tanh', 'topologia': [10, 8, 10, 4, 64, 64]}
Train on 500000 samples, validate on 50000 samples
Epoch 1/8
500000/500000 [==============================] - 5s 10us/step - loss: 0.6932 - acc: 0.5023 - val_loss: 0.6931 - val_acc: 0.5086
Epoch 2/8
500000/500000 [==============================] - 4s 9us/step - loss: 0.6931 - acc: 0.5063 - val_loss: 0.6931 - val_acc: 0.5047
Epoch 3/8
500000/500000 [==============================] - 4s 9us/step - loss: 0.6930 - acc: 0.5064 - val_loss: 0.6930 - val_acc: 0.5086
Epoch 4/8
500000/500000 [==============================] - 4s 9us/step - loss: 0.6930 - acc: 0.5081 - val_loss: 0.6930 - val_acc: 0.5086
Epoch 5/8
500000/500000 [==============================] - 4s 9us/step - loss: 0.6930 - acc: 0.5088 - val_loss: 0.6930 - val_acc: 0.5109
Epoch 6/8
500000/500000 [==============================] - 5s 9us/step - loss: 0.6929 - acc: 0.5091 - val_loss: 0.6930 - val_acc: 0.5069


In [12]:
resultado = pd.DataFrame(res)
resultado.sort_values(by=['score'],ascending=False)

,ativacao,batch_size,epochs,nrCamadas,optimizer,score,topologia
1,elu,128,8,8,sgd,0.517100,"[16, 3, 10, 5, 4, 128, 128, 2]"
3,tanh,64,8,2,adam,0.516647,"[16, 5]"
2,tanh,256,4,1,rmsprop,0.514918,[64]
0,tanh,128,8,6,sgd,0.512065,"[10, 8, 10, 4, 64, 64]"


In [3]:
parametros = selecaoHyperParametros(dicionario,neuronios)
for var in parametros:
    print()
    print(var)


{'nrCamadas': 1, 'epochs': 20, 'batch_size': 128, 'ativacao': 'sigmoid', 'topologia': [10]}

{'nrCamadas': 5, 'epochs': 8, 'batch_size': 512, 'ativacao': 'sigmoid', 'topologia': [8, 10, 4, 64, 64]}

{'nrCamadas': 1, 'epochs': 8, 'batch_size': 256, 'ativacao': 'tanh', 'topologia': [16]}

{'nrCamadas': 6, 'epochs': 4, 'batch_size': 256, 'ativacao': 'elu', 'topologia': [3, 10, 5, 4, 128, 128]}

{'nrCamadas': 3, 'epochs': 8, 'batch_size': 256, 'ativacao': 'linear', 'topologia': [2, 64, 16]}

{'nrCamadas': 4, 'epochs': 20, 'batch_size': 64, 'ativacao': 'relu', 'topologia': [5, 4, 16, 8]}

{'nrCamadas': 6, 'epochs': 10, 'batch_size': 64, 'ativacao': 'exponential', 'topologia': [16, 16, 3, 32, 128, 128]}

{'nrCamadas': 1, 'epochs': 4, 'batch_size': 256, 'ativacao': 'relu', 'topologia': [16]}

{'nrCamadas': 1, 'epochs': 10, 'batch_size': 256, 'ativacao': 'linear', 'topologia': [2]}

{'nrCamadas': 4, 'epochs': 20, 'batch_size': 256, 'ativacao': 'tanh', 'topologia': [2, 3, 128, 5]}
